## Introduktion

Den här lektionen kommer att ta upp:
- Vad funktionsanrop är och när det används
- Hur man skapar ett funktionsanrop med Azure OpenAI
- Hur man integrerar ett funktionsanrop i en applikation

## Lärandemål

Efter att ha slutfört den här lektionen kommer du att veta hur man och förstå:

- Syftet med att använda funktionsanrop
- Konfigurera Funktionsanrop med Azure Open AI-tjänsten
- Utforma effektiva funktionsanrop för din applikations användningsområde


## Förstå funktionsanrop

I den här lektionen vill vi bygga en funktion för vår utbildningsstartup som låter användare använda en chatbot för att hitta tekniska kurser. Vi kommer att rekommendera kurser som passar deras kunskapsnivå, nuvarande roll och teknikintresse.

För att genomföra detta kommer vi att använda en kombination av:
 - `Azure Open AI` för att skapa en chattupplevelse för användaren
 - `Microsoft Learn Catalog API` för att hjälpa användare att hitta kurser baserat på deras önskemål
 - `Function Calling` för att ta användarens fråga och skicka den till en funktion som gör API-anropet.

För att komma igång, låt oss titta på varför vi överhuvudtaget vill använda funktionsanrop:

print("Meddelanden i nästa förfrågan:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # hämta ett nytt svar från GPT där den kan se funktionssvaret


print(second_response.choices[0].message)


### Varför använda Function Calling

Om du har gått någon annan lektion i den här kursen, förstår du förmodligen redan styrkan med att använda Large Language Models (LLMs). Förhoppningsvis ser du också några av deras begränsningar.

Function Calling är en funktion i Azure Open AI Service som hjälper till att lösa följande begränsningar:
1) Konsekvent svarformat
2) Möjlighet att använda data från andra källor i en applikation i en chattkontext

Innan function calling var svaren från en LLM ostrukturerade och inkonsekventa. Utvecklare var tvungna att skriva komplex valideringskod för att kunna hantera varje variation av ett svar.

Användare kunde inte få svar som "Hur är vädret just nu i Stockholm?". Detta beror på att modellerna var begränsade till den tidpunkt då datan tränades.

Låt oss titta på exemplet nedan som illustrerar detta problem:

Säg att vi vill skapa en databas med studentdata så att vi kan föreslå rätt kurs till dem. Nedan har vi två beskrivningar av studenter som är väldigt lika i den data de innehåller.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Vi vill skicka detta till en LLM för att analysera datan. Detta kan senare användas i vår applikation för att skicka det till ett API eller lagra det i en databas.

Låt oss skapa två identiska uppmaningar där vi instruerar LLM:n om vilken information vi är intresserade av:


Vi vill skicka detta till en LLM för att analysera de delar som är viktiga för vår produkt. Så vi kan skapa två identiska uppmaningar för att instruera LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Efter att ha skapat dessa två uppmaningar kommer vi att skicka dem till LLM genom att använda `openai.ChatCompletion`. Vi lagrar uppmaningen i variabeln `messages` och tilldelar rollen till `user`. Detta är för att efterlikna ett meddelande från en användare som skrivs till en chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Även om uppmaningarna är desamma och beskrivningarna liknar varandra, kan vi få olika format på egenskapen `Grades`.

Om du kör cellen ovan flera gånger kan formatet vara `3.7` eller `3.7 GPA`.

Detta beror på att LLM tar ostrukturerad data i form av den skrivna uppmaningen och även returnerar ostrukturerad data. Vi behöver ha ett strukturerat format så att vi vet vad vi kan förvänta oss när vi lagrar eller använder denna data.

Genom att använda funktionella anrop kan vi säkerställa att vi får tillbaka strukturerad data. När vi använder funktionella anrop kör eller anropar LLM faktiskt inga funktioner. Istället skapar vi en struktur som LLM ska följa i sina svar. Vi använder sedan dessa strukturerade svar för att veta vilken funktion vi ska köra i våra applikationer.


![Funktionsanropsflödesdiagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.sv.png)


### Användningsområden för att använda funktionsanrop

**Anropa externa verktyg**  
Chattbottar är utmärkta på att ge svar på användarfrågor. Genom att använda funktionsanrop kan chattbottar använda meddelanden från användare för att utföra vissa uppgifter. Till exempel kan en student be chattbotten att "Skicka ett mejl till min lärare och säg att jag behöver mer hjälp med det här ämnet". Detta kan göra ett funktionsanrop till `send_email(to: string, body: string)`

**Skapa API- eller databasfrågor**  
Användare kan hitta information genom att använda naturligt språk som omvandlas till en formaterad fråga eller API-förfrågan. Ett exempel på detta kan vara en lärare som frågar "Vilka elever har gjort klart den senaste uppgiften", vilket kan anropa en funktion som heter `get_completed(student_name: string, assignment: int, current_status: string)`

**Skapa strukturerad data**  
Användare kan ta en textmassa eller en CSV-fil och använda LLM för att plocka ut viktig information från den. Till exempel kan en student omvandla en Wikipedia-artikel om fredsavtal för att skapa AI-flashkort. Detta kan göras genom att använda en funktion som heter `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Skapa ditt första funktionsanrop

Processen för att skapa ett funktionsanrop består av tre huvudsakliga steg:
1. Anropa Chat Completions API med en lista över dina funktioner och ett användarmeddelande
2. Läs modellens svar för att utföra en åtgärd, till exempel köra en funktion eller ett API-anrop
3. Gör ytterligare ett anrop till Chat Completions API med svaret från din funktion för att använda den informationen till att skapa ett svar till användaren.


![Flöde av ett funktionsanrop](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.sv.png)


### Element av ett funktionsanrop

#### Användarens inmatning

Första steget är att skapa ett användarmeddelande. Detta kan tilldelas dynamiskt genom att ta värdet från ett textfält, eller så kan du ange ett värde här. Om det är första gången du arbetar med Chat Completions API behöver vi definiera `role` och `content` för meddelandet.

`role` kan vara antingen `system` (skapar regler), `assistant` (modellen) eller `user` (slutanvändaren). Vid funktionsanrop sätter vi detta till `user` och ger ett exempel på en fråga.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Skapa funktioner.

Nu ska vi definiera en funktion och dess parametrar. Vi kommer att använda bara en funktion här som heter `search_courses`, men du kan skapa flera funktioner.

**Viktigt**: Funktioner inkluderas i systemmeddelandet till LLM och räknas in i det antal tillgängliga tokens du har.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitioner**

`name` - Namnet på funktionen som vi vill ska anropas.

`description` - Detta är en beskrivning av hur funktionen fungerar. Här är det viktigt att vara specifik och tydlig.

`parameters` - En lista över värden och format som du vill att modellen ska använda i sitt svar.

`type` - Datatypen som egenskaperna kommer att lagras i.

`properties` - Lista över de specifika värden som modellen kommer att använda i sitt svar.

`name` - Namnet på egenskapen som modellen kommer att använda i sitt formaterade svar.

`type` - Datatypen för denna egenskap.

`description` - Beskrivning av den specifika egenskapen.

**Valfritt**

`required` - Obligatorisk egenskap för att funktionsanropet ska kunna slutföras.


### Göra funktionsanropet
Efter att ha definierat en funktion behöver vi nu inkludera den i anropet till Chat Completion API:t. Det gör vi genom att lägga till `functions` i förfrågan. I det här fallet `functions=functions`.

Det finns också ett alternativ att sätta `function_call` till `auto`. Det innebär att vi låter LLM själv avgöra vilken funktion som ska anropas baserat på användarens meddelande istället för att vi bestämmer det själva.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Nu ska vi titta på svaret och se hur det är formaterat:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Du kan se att namnet på funktionen anropas och från användarens meddelande kunde LLM hitta data för att fylla i argumenten till funktionen.


## 3. Integrera funktionsanrop i en applikation.

Efter att vi har testat det formaterade svaret från LLM kan vi nu integrera detta i en applikation.

### Hantera flödet

För att integrera detta i vår applikation, låt oss följa dessa steg:

Först gör vi anropet till Open AI-tjänsterna och sparar meddelandet i en variabel som heter `response_message`.


In [ ]:
response_message = response.choices[0].message

Nu ska vi definiera funktionen som kommer att anropa Microsoft Learn API för att hämta en lista över kurser:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Som en rekommenderad metod kommer vi sedan att se om modellen vill anropa en funktion. Därefter skapar vi en av de tillgängliga funktionerna och matchar den mot den funktion som anropas.
Vi tar sedan argumenten från funktionen och kopplar dem till argumenten från LLM.

Slutligen lägger vi till meddelandet om funktionsanropet och de värden som returnerades av meddelandet `search_courses`. Detta ger LLM all information den behöver för att
svara användaren med naturligt språk.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Kodutmaning

Bra jobbat! För att fortsätta din inlärning om Azure Open AI Function Calling kan du bygga: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Fler parametrar för funktionen som kan hjälpa användare att hitta fler kurser. Du hittar tillgängliga API-parametrar här:
 - Skapa ett annat funktionsanrop som tar emot mer information från användaren, till exempel deras modersmål
 - Skapa felhantering när funktionsanropet och/eller API-anropet inte returnerar några lämpliga kurser



---

**Ansvarsfriskrivning**:  
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Vi strävar efter noggrannhet, men var medveten om att automatiska översättningar kan innehålla fel eller brister. Det ursprungliga dokumentet på dess originalspråk ska betraktas som den auktoritativa källan. För kritisk information rekommenderas professionell mänsklig översättning. Vi tar inget ansvar för eventuella missförstånd eller feltolkningar som uppstår vid användning av denna översättning.
